# 📖 Capítol 4 - Algorismes i Text

Una seqüència genètica és una cadena (string) formada per caràcters d'un alfabet de quatre lletres: A, T, G, C, anomenats **bases**, que corresponen a les macromolècules de l'**ADN**. Un **gen** és una seqüència ordenada de bases i el **genoma** és la concatenació de tots els gens.

Cada cèl·lula produïda pel cos rep una còpia del genoma, però sovint aquesta còpia és alterada. Les possibles alteracions que es poden produir són, entre d'altres, la substitució d'una base per una altra o la pèrdua d'una base.

### ✍️ Exercici 1. Funció dna 

Fes una funció, anomenada "dna", basada en l'algorisme de Levensthein, que busqui dins d'una seqüència genètica una cadena genètica passada per paràmetre.

Aquesta funció ha de retornar la línia del fitxer on comença la cadena més semblant i la distància entre la cadena d'entrada i la cadena més semblant.

<span style="color:Blue">El càlcul  de la distància d'un patró al *substring* més semblant d'un text es pot fer amb l'algorisme de Levenshtein. L'única diferència és que s'ha d'inicialitzar la primera fila amb zeros i que la distància d'edició serà el valor mínim de l'última fila de la matriu de costos. També has de tenir en compte els costos en la inicialització de la primera columna.</span>


La seqüència genètica que farem servir és la del cromosoma 2 humà (fitxer HUMAN-DNA.txt).

Les primeres línies d'aquest fitxer tenen aquesta forma:

CCCATCTCTTTCTCATTCCTTGGTTGAGAACACGAACTTCAGGACTTGCCTCACACTAGGGCCCATTCTT
TGTTTCCCAGAAAGAAGAGGCTCTCCACACAGAGTCCCATGTACACCAGGCTGTCAACAAACATGAATTG
AATGAAGGAGTGGATGGTTGGGTGGAAGTGATTTAAGAAATCCTAACTGGGGAATTTCACTGGAAACTTA

En programar aquesta funció, cal que tinguis en compte que, en aplicacions bioinformàtiques, els costos de les operacions d'edició són lleugerament diferents dels que hem vist fins ara:

+ Per a un salt o inserció (al patró o al text), el cost és 2
+ Per a una substitució, el cost és 1
+ Quan hi ha correspondència, el cost és 0.

Usa els següents patrons:

In [ ]:
def levenstheinsmithwaterman(patro, text, dlt=2, insr=2, subs=1):

    m, n = len(patro), len(text)
    dp = [[0]*(n+1) for _ in range(m+1)]

   
    for i in range(1, m+1):
        dp[i][0] = dp[i-1][0] + dlt

   
    for i in range(1, m+1):
        for j in range(1, n+1):
            cost_subs = 0 if patro[i-1] == text[j-1] else subs
            dp[i][j] = min(
                dp[i-1][j] + dlt,         # eliminació
                dp[i][j-1] + insr,        # inserció
                dp[i-1][j-1] + cost_subs  # sustitució
            )

    # Distancia minima ultima fila
    min_distance = min(dp[m])
    return min_distance


def dna(patro, fitxer="HUMAN-DNA.txt"):
    
    best_line = -1
    best_distance = float("inf")

    with open(fitxer, "r") as f:
        for idx, line in enumerate(f, start=1):
            seq = line.strip()
            dist = levenstheinsmithwaterman(patro, seq)
            if dist < best_distance:
                best_distance = dist
                best_line = idx

    return best_line, best_distance


In [2]:
assert dna('AGATACATTAGACAATAGAGATGTGGTC') == (32, 11)
assert dna('GTCAGTCTGGCCTTGCCATTGGTGCCACCA') == (352, 11)
assert dna('TACCGAGAAGCTGGATTACAGCATGTACCATCAT') == (233, 13)

Si a més de saber la distància volem saber quins canvis hi ha hagut haurem de modificar els anteriors algorismes per guardar els canvis a cada pas i un cop trobada la distància mínima desfer els passos i anar apuntant els canvis.

Recordem que hi pot haver 4 tipus de canvis

+ I: Insertion
+ D: Deletion
+ S: Substitution
+ C: Coincidence (no hi ha canvis)



### ✍️ Reescriu les anteriors funcions per registrar els canvis i per mostrar-los al final.

In [20]:
def dna(patro, fitxer='HUMAN-DNA.txt'):
    """
    Cerca el patró en un fitxer de DNA amb l'algorisme Levenshtein/Smith-Waterman.
    
    Parameters
    ----------
    patro: string
        Patró de DNA que volem buscar.
    fitxer: string (default 'HUMAN-DNA.txt')
        Fitxer amb les seqüències de DNA.
    
    Returns
    -------
    tuple: (millor_linia, (inici_text, final_text), distancia_minima, sequencia_moviments)
        Retorna la línia amb la millor coincidència, la posició inicial i final dins la seqüència,
        la distància mínima i la seqüència de moviments aplicats al patró.
    """

    millor_linia = -1
    millor_resultat = None
    distancia_minima_global = float("inf")

    with open(fitxer, "r") as f:
        for linia, seq in enumerate(f, start=1):
            seq = seq.strip()  # Elimina espais o salts de línia
            (inici, final), dist, movs, mat, seq_mov = levenstheinsmithwaterman(patro, seq)
            
            # Comprovem si aquesta línia és millor
            if dist < distancia_minima_global:
                distancia_minima_global = dist
                millor_linia = linia
                millor_resultat = (inici, final, dist, movs, mat, seq_mov)

    # Desem les dades del millor resultat
    inici_text, final_text, distancia_minima, matriu_moviments, matriu_distancia, sequencia_moviments = millor_resultat


    return (millor_linia, (inici_text, final_text), distancia_minima, sequencia_moviments)


In [18]:
assert dna("CTGGTACCAGCTGTATTAGC") == (729,(11, 30), 6, ['C',  'C',  'C',  'C',  'C',  'C',  'C',  'S',  'C',  'S',  'C',  'S',  'S',  'S',  'C',  'C',  'S',  'C',  'C',  'C'])
assert dna("TCGTCATAAACCGCTGTGCC") == (213,(12, 31), 7, ['S',  'C',  'S',  'C',  'C',  'C',  'C',  'C',  'C',  'C',  'C',  'C',  'S',  'S',  'C',  'C',  'S',  'S',  'C',  'S'])
assert dna("TATACAAACGGAGTAGCTGT") == (286, (5, 24), 6, ['C',  'C',  'C',  'C',  'S',  'C',  'S',  'C',  'C',  'S',  'C',  'S',  'S',  'C',  'C',  'C',  'S',  'C',  'C',  'C'])
assert dna("AGGCGTAAGTCTTACGTATA") == (6, (41, 60), 7, ['C',  'S',  'C',  'S',  'S',  'C',  'C',  'C',  'C',  'C',  'C',  'S',  'C',  'S',  'S',  'C',  'S',  'C',  'C',  'C'])
assert dna("AACGGCATAGCCTGCAAGAG") == (434, (41, 60), 5, ['C',  'C',  'S',  'C',  'C',  'C',  'C',  'S',  'C',  'S',  'C',  'C',  'C',  'C',  'C',  'C',  'C',  'S',  'C',  'S'])


AssertionError: 